# Assigment 3 - DDG
## Edoardo Vassallo - S4965918

In [44]:
import igl
import numpy as np
import meshplot as mp
from collections import deque
from typing import List
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp

## 0 - Load and Display Mesh

In [45]:
v, f = igl.read_triangle_mesh("data/bunny.off")
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

## 1 - Vertex Normals

### 1.1 - Standard Vertex Normals

In [46]:
#Standard face normal
n = np.zeros_like(v)

# by default computes as uniform, specified just for clarity
n = igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)

# compute normals for each face
# accumulate the normal to the entries of n corresponding to the face's vertices
# normalize all entries
# save for later use

mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### 1.2 - Area Weighted Normals

In [47]:
#Area-weighted face normal
n = np.zeros_like(v)

n = igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA)

#same as above, but faces' normals shall not be normalized prior to accumulate (cross product includes area)

mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### 1.3 - Mean Curvature Normals

In [48]:
#Compute Laplacian matrix
#Compute normal direction from mean curvature formula
#If the norm if smaller then eps, substitute with the saved standard normal
#Normalize
#Adjust directions by using dot product with standard normal


def compute_mean_curvature_normals(v, f, eps=1e-6):
    # Compute cotangent-weighted Laplacian matrix
    L = igl.cotmatrix(v, f)
    # Compute mass matrix
    M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    # Compute mean curvature normals HN = inv(M) * (L.dot(v)) 
    # [https://libigl.github.io/libigl-python-bindings/tut-chapter1/#curvature-directions]
    minv = sp.diags(1 / M.diagonal())
    HN = -minv.dot(L.dot(v))
    
    # Prepare fallback area-weighted normals
    fallback = igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA)
    # Normalize and threshold
    N = np.zeros_like(HN)
    lengths = np.linalg.norm(HN, axis=1)
    for i in range(HN.shape[0]):
        if lengths[i] > eps:
            N[i] = HN[i] / lengths[i]
        else:
            N[i] = fallback[i]
        
    # fix signs
    sign_flip = np.sign((N * fallback).sum(axis=1))[:, None]
    return sign_flip * N

In [49]:
#Mean-curvature normal
n = np.zeros_like(v)

# The result is very noisy
n = compute_mean_curvature_normals(v, f)
mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### 1.4 - PCA Normals

In [50]:
## AUXILLARY FUNCTION:
# Compute k-ring for each vertex
def compute_vertex_k_ring(V, F, k = 5):

    n = V.shape[0]
    # initialize output
    nbrs: List[List[int]] = [[] for _ in range(n)]

    # Get vertex-triangle adjacency
    v2t_flat, v2t_ptr = igl.vertex_triangle_adjacency(F, n)

    # for each vertex
    for vi in range(n):
        # use BFS to find k-ring neighbors
        seen = {vi}
        queue = deque([(vi, 0)])
        
        while queue:
            vj, d = queue.popleft()
            # if we are inside the k-ring,
            if d < k:
                #  we add new neighbors to the set
                s, e = v2t_ptr[vj], v2t_ptr[vj+1]
                for t in v2t_flat[s:e]:
                    for vk in F[t]:
                        if vk not in seen:
                            seen.add(vk)
                            queue.append((vk, d+1))
        nbrs[vi] = list(seen)

    return nbrs

In [51]:
def compute_pca_normals(v, f, k=16):

    N = np.zeros_like(v)
    all_neighbours = compute_vertex_k_ring(v, f, k=k)
    # for each vertex
    for i, vi in enumerate(v):

        # get neighbours
        nbrs = all_neighbours[i]
        if len(nbrs) < 3:
            N[i] = np.array([0.0, 0.0, 1.0])
            continue
        pts = v[nbrs]

        # center the points
        pts -= pts.mean(axis=0)

        # compute PCA
        cov = np.cov(pts.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        
        # smallest eigenvector is the normal
        normal = eigvecs[:, np.argmin(eigvals)]
        N[i] = normal

    # fix signs
    stdN = igl.per_vertex_normals(v, f)
    sign_flip = np.sign((N * stdN).sum(axis=1))[:, None]
    return sign_flip * N

In [52]:
#PCA normal

n = np.zeros_like(v)
k = 3

n = compute_pca_normals(v, f, k)
#Define function to find kNN of a vertex (either by breadth-first expansion, or by exhaustive search)

#For each vertex:
#   find its kNN's
#   build the covariance matrix
#   find its eigenvectors by eigen decomposition
#   set normal from eigenvector corresponding to minimum eigenvalue
#Adjust directions as above

mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### 1.5 - Quadric Fitting Normals

In [53]:
def compute_quadratic_normals(V, F, k=16):
    N = np.zeros_like(V)
    stdN = igl.per_vertex_normals(V, F)
    all_neighbours = compute_vertex_k_ring(V, F, k=k)
    # for each vertex
    for i, vi in enumerate(V):

        # get neighbours
        nbrs = all_neighbours[i]

        if len(nbrs) < 3:
            N[i] = np.array([0.0, 0.0, 1.0])
            continue

        pts = V[nbrs]
        
        # center points to mean of the neighbourhood
        pts -= pts.mean(axis=0)

        # compute PCA
        cov = np.cov(pts.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        e1, e2 = eigvecs[:, 1], eigvecs[:, 2]
        e3 = eigvecs[:, 0] # smallest eigenvector is the height

        # compute coordinates (u, v, h) in the local frame
        coords = np.dot(pts, np.vstack([e1, e2, e3]).T)
        u, v, h = coords[:,0], coords[:,1], coords[:,2]

        # we'll use as quadratic function h = a*u^2 + b*u*v + c*v^2 + d*u + e*v + f
        A = np.column_stack([u*u, u*v, v*v, u, v, np.ones_like(u)])
        # we want to fit the quadratic function to the height h
        coeffs, *_ = np.linalg.lstsq(A, h, rcond=None)
        # coeffs = [a, b, c, d, e, f]
        # we only care about d and e, because they are 
        # the derivatives in (0, 0), the vertex
        _, _, _, fu, fv, _ = coeffs

        # compute the tangent plane basis
        tu = e1 + fu * e3
        tv = e2 + fv * e3
        # the normal is the cross product of the tangent basis
        N[i] = np.cross(tu, tv)
        N[i] /= np.linalg.norm(N[i])

    # fix signs
    sign_flip = np.sign((N * stdN).sum(axis=1))[:, None]
    return sign_flip * N

In [54]:
#Quadratic fitting normal

n = np.zeros_like(v)
k = 3


n = compute_quadratic_normals(v, f, k)

#For each vertex:
#   find kNN's and compute PCA as before
#   express all neighbors in a local ref framewith origin at the vertex and axes aligned to the PCA
#   fit a quadratic polynomial to such data
#   compute the components of the Jacobian at the origin and express them in the global coordinate system
#   compute the normal as orthogonal to the Jacobian's components
#Adjust directions as above

mp.plot(v, f, n=n, shading={"flat": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

## 2 - Curvature

### 2.1 - From Gaussian Curvature

In [55]:
#Obtain Gaussian curvature k from related igl function 
k = igl.gaussian_curvature(v, f)          # integrated K
# K      = Minv @ K_int   

#Compute mean curvature h from Laplacian as above
L = igl.cotmatrix(v, f)
M      = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
Minv   = sp.diags(1.0 / M.diagonal())
hn = -Minv.dot(L.dot(v))
h = np.linalg.norm(hn, axis=1)

#Compute principal curvatures k1 and k2 from k and h
sqrt = np.sqrt(np.maximum(h*h - k, 0.0))
k1 = h - sqrt
k2 = h + sqrt

# Results are very noisy, almost unrecognizable
# different colormaps have been used for each variable
# they are kept the same with the next section for comparison
mp.plot(v, f, k,  shading={"flat": False, "colormap" : "brg"})
mp.plot(v, f, h,  shading={"flat": False, "colormap" : "cubehelix"})
mp.plot(v, f, k1, shading={"flat": False, "colormap" : "hsv"})
mp.plot(v, f, k2, shading={"flat": False, "colormap" : "binary"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

### 2.2 - From Principal Curvatures

In [56]:
#Obtain principal curvatures k1 and k2 and their directions v1 and v2 from related igl function
v1, v2, k1, k2 = igl.principal_curvature(v, f)
#Compute Gaussian curvature k and mean curvature h from k1 and k2
k = k1 * k2
h = 0.5*(k1 + k2)

# plot Gaussian curvature
mp.plot(v, f, k, shading={"wireframe": False, "colormap" : "brg"})

# plot Mean Curvature
mp.plot(v, f, h, shading={"wireframe": False, "colormap" : "cubehelix"})

# plot Principal Curvatures
p = mp.plot(v, f, k1, shading={"wireframe": False, "colormap" : "hsv"}, return_plot=True)
avg = igl.avg_edge_length(v, f) / 2.0
p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})

p = mp.plot(v, f, k2, shading={"wireframe": False, "colormap" : "binary"}, return_plot=True)
avg = igl.avg_edge_length(v, f) / 2.0
p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

2

## 3 - Smoothing with the Laplacian

### 3.1 Explicit Smoothing

In [57]:
## AUXILLARY FUNCTION

# compute uniform Laplacian matrix
def normalized_uniform_laplacian(f):
    # build unweighted adjacency A
    A = igl.adjacency_matrix(f)

    # degree per vertex
    deg = np.array(A.sum(axis=1)).ravel()
    # Build W where W[i,j] = 1/deg(i) if A[i,j]==1
    inv_deg = 1.0 / deg
    D_inv = sp.diags(inv_deg)  
    # each row i of A scaled by 1/deg(i)
    W = D_inv @ A                                  
    # Now set diagonal entries to -1
    n = A.shape[0]
    L_uni = W - sp.eye(n)

    return L_uni

In [58]:
# Explicit laplacian
ll = 0.1  # speed factor
it = 1000      # iterations

# MAIN FORMULA
# p_t+1 = (I + ll * m^-1 x L )p_t

# Uniform weights
L_uni = normalized_uniform_laplacian(f)

# Cotangent weights
# Compute stiffness matrix (cotmatrix) and mass matrix
L = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

# Build matrix for forward Euler

# Invert the diagonal of the mass matrix
# Minv = m^-1
Minv = sp.diags(1 / m.diagonal())

# Build the operator 
# M^-1 * L
L_tan = Minv @ L

# Run forward Euler on a copy vi of v
vi = v.copy()

# Run forward Euler iterations
# WARN: choose between L_uni and L_tan
# uniform weights seem to be way slower than cotan
for _ in range(it):
    vi += ll * (L_uni @ vi)
    #vi += ll * (L_tan @ vi)

# Plot the result
p = mp.plot(vi, f)

# Experiment with different speeds and iterations

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.020215…

### 3.2 - Implicit Smoothing

In [59]:
# Implicit laplacian

ll = 0.0001  # speed factor

# MAIN FORMULA
# (m - ll * L ) p_t+1 = m * p_t 

# Compute stiffness matrix (cotmatrix) and mass matrix
L = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

# Build matrix and right hand side for backward Euler

# Left hand side
# A = M - ll * L
A = m - ll * L

# Right-hand side: 
# B = M * v
B = m @ v

# Solve the linear system 
# A * vi = B
vi = spsolve(A, B)

# Plot the result
p = mp.plot(vi, f)

# Experiment with different parameters

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.017820…